## Use of PULP to imply Dc load curtailment in IEEE RTS 24 bus system


In [3]:
import pandas as pd
import numpy as np

from pulp import*
from random import*


#Reading the datas from CSV file

B_matrix=pd.read_csv('Bmatrix.csv')
bA_matrix=pd.read_csv('bA.csv')
gen_data=pd.read_csv('f2.csv')
line_data=pd.read_csv('f3v2.csv')

In [5]:

#Defining the constants required
Nb=24  
Nt=38
sum_C=B_matrix.sum(axis=0)
M=np.zeros(24)
for i in range (0,24):
    M[i]=sum_C[i]

Demand=gen_data['Demand']
D=np.zeros(24)
for i in range (0,24):
      D[i]=Demand[i]/100

# for i in range (0,24):
#      D[i]=2

Gmax=[1.92,1.92,0,0,0,0,3,0,0,0,0,0,5.91,0,2.15,1.55,0,4,0,0,4,3,6.6,0]    
Gmin=[0.624,0.624,0,0,0,0,0.75,0,0,0,0,0,2.07,0,0.663,0.543,0,1,0,0,1,0.6,2.486,0]

Lmax=line_data['rateA']
Fmax=np.zeros(38)
for i in range(0,38):
    Fmax[i]=Lmax[i]/100

    
# A new LP problem in PULP
load_curtailment = LpProblem("test", LpMinimize)

#Defining optimization variables
G = LpVariable.matrix("G", list(range(Nb)),cat=' LpInteger')
G[2],G[3],G[4],G[5],G[7],G[8],G[9],G[10],G[11],G[13],G[16],G[18],G[19],G[23]=0,0,0,0,0,0,0,0,0,0,0,0,0,0

I= LpVariable.matrix("I", list(range(Nb)),1,1,cat=' LpInteger')

delta=LpVariable.matrix("del",list(range(Nb)),cat='LpInteger')
delta[12]=0  # bus 13 is slack bus


#Defining constraints
for i in range(Nb):
        if i!=2 and i!=3 and i!=4 and i!=5 and i!=7 and i!=8 and i!=9 and i!=10 and i!=11 and i!=13 and i!=16 and i!=18 and i!=19 and i!=23  :
            load_curtailment += G[i]>=Gmin[i]
            load_curtailment += G[i]<=Gmax[i]   
        load_curtailment += I[i]>=1   
        load_curtailment += G[i]+lpDot(delta,B_matrix.loc[i,:])<=D[i]
        load_curtailment += -G[i]-lpDot(delta,B_matrix.loc[i,:])<=0
        
for j in range(Nt):        
        load_curtailment +=lpDot(bA_matrix.loc[i,:],delta)<=Fmax[i]
        load_curtailment += -lpDot(bA_matrix.loc[i,:],delta)<=Fmax[i]
        
        
#Defining objective function
load_curtailment += -lpSum(G)-lpDot(M,delta)+lpDot(I,D)

load_curtailment.solve()
load_curtailment.writeLP("DC curtailment.lp")



# Printing the values of problem
print("Status:", LpStatus[load_curtailment.status])

print("Load curtailment value:", load_curtailment.objective.value())

for v in load_curtailment.variables():
 	print(v.name, "=", v.varValue)
    
# print('\n')
# v=load_curtailment.objective.value()+lpSum(D)
# print('Actual value',v)
# print('Total load',lpSum(D))


Status: Optimal
Load curtailment value: -2.1552590601103816e-08
G_0 = 1.8646625
G_1 = 1.92
G_12 = 5.91
G_14 = 0.663
G_15 = 0.543
G_17 = 1.0
G_20 = 4.0
G_21 = 3.0
G_22 = 6.6
G_6 = 3.0
I_0 = 1.0
I_1 = 1.0
I_10 = 1.0
I_11 = 1.0
I_12 = 1.0
I_13 = 1.0
I_14 = 1.0
I_15 = 1.0
I_16 = 1.0
I_17 = 1.0
I_18 = 1.0
I_19 = 1.0
I_2 = 1.0
I_20 = 1.0
I_21 = 1.0
I_22 = 1.0
I_23 = 1.0
I_3 = 1.0
I_4 = 1.0
I_5 = 1.0
I_6 = 1.0
I_7 = 1.0
I_8 = 1.0
I_9 = 1.0
del_0 = 0.1937764
del_1 = 0.19262699
del_10 = 0.12947418
del_11 = 0.076759224
del_13 = 0.1549294
del_14 = 0.11691114
del_15 = 0.10315259
del_16 = 0.053915537
del_17 = 0.049507559
del_18 = 0.087392121
del_19 = -0.011301822
del_2 = 0.26048541
del_20 = -0.01876768
del_21 = -0.1140311
del_22 = -0.092782881
del_23 = 0.17178231
del_3 = 0.24420659
del_4 = 0.24037794
del_5 = 0.28085968
del_6 = 0.10728153
del_7 = 0.21473153
del_8 = 0.20968489
del_9 = 0.22638217


In [44]:
if load_curtailment.objective.value()>0:
    print("There is load curtailment of ",load_curtailment.objective.value(),"MW in pu")
else:
    print("There is no load curtailment beacuse the value is=",load_curtailment.objective.value())
    

There is no load curtailment beacuse the value is= -2.1552590601103816e-08


In [1]:
len(Gmax)

NameError: name 'Gmax' is not defined